<a href="https://colab.research.google.com/github/VKSi/2021_03_GB_NLP/blob/main/HW5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 2

много дополнительных датасетов на русском языке

https://natasha.github.io/corus/  
https://github.com/natasha/corus

мы будем использовать данные http://ai-center.botik.ru/Airec/index.php/ru/collections/28-persons-1000

**Проверить насколько хорошо работает NER**

1. взять нер из nltk
2. проверить deeppavlov
3. написать свой нер попробовать разные подходы (с доп информацией без) так же с учётом соседей и без них
4. сделать выводы по вашим экспериментам какой из подходов успешнее справляется

при обучении своего нера незабудьте разделить выборку

In [ ]:
!pip install corus

In [ ]:
import corus

In [ ]:
!wget http://ai-center.botik.ru/Airec/ai-resources/Persons-1000.zip

--2021-04-18 18:29:30--  http://ai-center.botik.ru/Airec/ai-resources/Persons-1000.zip
Resolving ai-center.botik.ru (ai-center.botik.ru)... 95.129.138.2
Connecting to ai-center.botik.ru (ai-center.botik.ru)|95.129.138.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3363777 (3.2M) [application/zip]
Saving to: ‘Persons-1000.zip.1’

Persons-1000.zip.1  100%[===================>]   3.21M  1.17MB/s    in 2.8s    

2021-04-18 18:29:33 (1.17 MB/s) - ‘Persons-1000.zip.1’ saved [3363777/3363777]



In [ ]:
!ls

Persons-1000.zip  Persons-1000.zip.1  sample_data


In [ ]:
path = 'Persons-1000.zip'
records = corus.persons.load_persons(path)
rec = next(records)

In [ ]:
rec

PersonsMarkup(
    text='Россия рассчитывает на конструктивное воздействие США на Грузию\r\n\r\n04/08/2008 12:08\r\n\r\nМОСКВА, 4 авг - РИА Новости. Россия рассчитывает, что США воздействуют на Тбилиси в связи с обострением ситуации в зоне грузино-осетинского конфликта. Об этом статс-секретарь - заместитель министра иностранных дел России Григорий Карасин заявил в телефонном разговоре с заместителем госсекретаря США Дэниэлом Фридом.\r\n\r\n"С российской стороны выражена глубокая озабоченность в связи с новым витком напряженности вокруг Южной Осетии, противозаконными действиями грузинской стороны по наращиванию своих вооруженных сил в регионе, бесконтрольным строительством фортификационных сооружений", - говорится в сообщении.\r\n\r\n"Россия уже призвала Тбилиси к ответственной линии и рассчитывает также на конструктивное воздействие со стороны Вашингтона", - сообщил МИД России. ',
    spans=[PersonsSpan(
         id=1,
         start=308,
         stop=324,
         value='ГРИГОРИЙ КАР

In [ ]:
print(rec.text)

Россия рассчитывает на конструктивное воздействие США на Грузию

04/08/2008 12:08

МОСКВА, 4 авг - РИА Новости. Россия рассчитывает, что США воздействуют на Тбилиси в связи с обострением ситуации в зоне грузино-осетинского конфликта. Об этом статс-секретарь - заместитель министра иностранных дел России Григорий Карасин заявил в телефонном разговоре с заместителем госсекретаря США Дэниэлом Фридом.

"С российской стороны выражена глубокая озабоченность в связи с новым витком напряженности вокруг Южной Осетии, противозаконными действиями грузинской стороны по наращиванию своих вооруженных сил в регионе, бесконтрольным строительством фортификационных сооружений", - говорится в сообщении.

"Россия уже призвала Тбилиси к ответственной линии и рассчитывает также на конструктивное воздействие со стороны Вашингтона", - сообщил МИД России. 


In [ ]:
rec.spans

[PersonsSpan(
     id=1,
     start=308,
     stop=324,
     value='ГРИГОРИЙ КАРАСИН'
 ), PersonsSpan(
     id=2,
     start=387,
     stop=402,
     value='ДЭНИЭЛ ФРИД'
 )]

In [ ]:
!pip install razdel

In [ ]:
from razdel import tokenize

In [ ]:
records = corus.persons.load_persons(path)
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        is_person = False
        for person in rec.spans:
            if (token.start >= person.start) and (token.stop <= person.stop):
                is_person = True
                break
        words.append([token.text, 'PERSON' if is_person else 'NO_PERSON'])
    words_docs.extend(words)

Подстрою предложенный подход непосредственно под наше залание. Нам достаточно отслеживать множество персон:

In [ ]:
records = corus.persons.load_persons(path)
person_set = set([])
for ix, rec in enumerate(records):
  for token in tokenize(rec.text):
    is_person = False
    for person in rec.spans:
        if (token.start >= person.start) and (token.stop <= person.stop):
            is_person = True
            break
    if is_person:
      person_set.add(token.text)

теперь у нас есть слово и тег персона или нет, уже можно что-то делать(пробовать какие-то мл алгоритмы)  
*дополнительно попробуйте проанализировать все ли токены если рассматривать униграмный подход относятся к тегам персона с логической точки зрения

In [ ]:
len(person_set)

5294

In [ ]:
person_set.intersection(set(['Хаммарберг']))

{'Хаммарберг'}

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_ru')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [ ]:
records = corus.persons.load_persons(path)
nltk_person_set = set([])
for rec in records:
  nltk_person_set = nltk_person_set.union({' '.join(c[0] for c in chunk) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(rec.text))) if (hasattr(chunk, 'label') and chunk.label() == 'PERSON') })

In [ ]:
len(nltk_person_set)

4882

In [ ]:
nltk_person_set.intersection(set(['Хаммарберг']))

{'Хаммарберг'}

In [ ]:
print(f'recall = {len(nltk_person_set.intersection(person_set))/len(person_set)*100:.2f}%')

recall = 18.28%


In [ ]:
len(nltk_person_set - person_set)/len(person_set)

0.7393275406120136

In [ ]:
len(person_set - nltk_person_set)/len(person_set)

0.8171514922553834

 Совпало только 18% сущностей, определенных нами ранее как "персоны"

In [ ]:
   # установка deeppavlov

!pip uninstall -y tensorflow tensorflow-gpu
!pip install numpy scipy librosa unidecode inflect librosa transformers
!pip install deeppavlov

Uninstalling tensorflow-1.15.2:
  Successfully uninstalled tensorflow-1.15.2


In [ ]:
!python -m deeppavlov install squad_bert
!python -m deeppavlov install ner_ontonotes

2021-04-18 18:31:32.192 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'squad_bert' as '/usr/local/lib/python3.7/dist-packages/deeppavlov/configs/squad/squad_bert.json'
  Using cached https://files.pythonhosted.org/packages/5b/81/84fb7a323f9723f81edfc796d89e89aa95a9446ed7353c144195b3a3a3ba/tensorflow-1.15.2-cp37-cp37m-manylinux2010_x86_64.whl
  Cloning https://github.com/deepmipt/bert.git (to revision feat/multi_gpu) to /tmp/pip-req-build-2e6emph_
  Running command git clone -q https://github.com/deepmipt/bert.git /tmp/pip-req-build-2e6emph_
  Created wheel for bert-dp: filename=bert_dp-1.0-cp37-none-any.whl size=23581 sha256=ac996cff4d8b25d8f9ff3941ef5c0616b6bd690d3813569ee2478675573d9ae0
  Stored in directory: /tmp/pip-ephem-wheel-cache-21dg43_d/wheels/1e/41/94/886107eaf932532594886fd8bfc9cb9d4db632e94add49d326
Successfully built bert-dp
2021-04-18 18:31:59.61 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'ner_ontonotes' as '/usr/local

In [ ]:
import deeppavlov
from deeppavlov import configs, build_model

In [ ]:
ner_model = build_model(configs.ner.ner_ontonotes_bert_mult, download=True)
rus_document = "Нью-Йорк, США, 30 апреля 2020, 01:01 — REGNUM В администрации президента США Дональда Трампа планируют пройти все этапы создания вакцины от коронавируса в ускоренном темпе и выпустить 100 млн доз до конца 2020 года, передаёт агентство Bloomberg со ссылкой на осведомлённые источники"
ner_model([rus_document])

2021-04-18 18:32:21.757 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip download because of matching hashes
2021-04-18 18:32:39.90 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/ner_ontonotes_bert_mult_v1.tar.gz download because of matching hashes
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


2021-04-18 18:32:41.430 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_ontonotes_bert_mult/tag.dict]











The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API




Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use sta

2021-04-18 18:33:16.323 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/ner_ontonotes_bert_mult/model]



INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_ontonotes_bert_mult/model


[[['Нью',
   '-',
   'Йорк',
   ',',
   'США',
   ',',
   '30',
   'апреля',
   '2020',
   ',',
   '01',
   ':',
   '01',
   '—',
   'REGNUM',
   'В',
   'администрации',
   'президента',
   'США',
   'Дональда',
   'Трампа',
   'планируют',
   'пройти',
   'все',
   'этапы',
   'создания',
   'вакцины',
   'от',
   'коронавируса',
   'в',
   'ускоренном',
   'темпе',
   'и',
   'выпустить',
   '100',
   'млн',
   'доз',
   'до',
   'конца',
   '2020',
   'года',
   ',',
   'передаёт',
   'агентство',
   'Bloomberg',
   'со',
   'ссылкой',
   'на',
   'осведомлённые',
   'источники']],
 [['B-GPE',
   'I-GPE',
   'I-GPE',
   'O',
   'B-GPE',
   'O',
   'B-DATE',
   'I-DATE',
   'I-DATE',
   'O',
   'B-TIME',
   'I-TIME',
   'I-TIME',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-GPE',
   'B-PERSON',
   'I-PERSON',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-MONEY',
   'I-MONEY',
   'I-MONEY',
   'O',
   'B-DATE',
   'I-DA

In [ ]:
records = corus.persons.load_persons(path)
pavlova_person_set = set([])
for rec in records:
  try:
    ner = ner_model([rec.text])
    pv_txt, pv_lbls = ner_model([rec.text])[0], ner_model([rec.text])[1]
    
    for ind, lbl in enumerate(pv_lbls[0]):
      if 'PERSON' in str(lbl):
        person = pv_txt[0][ind]
        pavlova_person_set.add(person)
  except:
    continue


In [ ]:
pavlova_person_set.intersection(set(['Хаммарберг']))

In [ ]:
print(f'recall = {len(pavlova_person_set.intersection(person_set))/len(person_set)*100:.2f}%')

recall = 61.96%


In [ ]:
print(f'FP = {len(pavlova_person_set - person_set)/len(person_set):.2f}%')

FP = 0.03%


In [ ]:
print(f'FN = {len(person_set - pavlova_person_set)/len(person_set):.2f}%')

FN = 0.38%


Результат гораздо лучше, чем у nltk

Третью часть сделать уже не успеваю (((